Import Libraries

In [161]:
import requests
from bs4 import BeautifulSoup as bs4

import time
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', ResourceWarning)

import pandas as pd

Scrape card list from Yu-gi-oh database and get html to collect card name.
Target deck is "DragonLink_PureVer1.0", which author have used it in master duel.

遊戯王の公式DBに登録したデッキ紹介ページからHTMLを読み込みます。
今回、分析するデッキはドラゴンリンクデッキで、自身がマスターデュエルで使ってるデッキです。

Deck link ->
(JP)
http://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=ja
(ENG)
https://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=en

In [162]:
URL = 'http://www.db.yugioh-card.com/yugiohdb/member_deck.action?cgid=7d079deb81cbfbb48026ba3600c52d62&dno=4&request_locale=ja'
res = requests.get(URL)
if res.status_code != requests.codes.ok:
    print('Error')
    exit(1)
soup = bs4(res.text)
soup.head() #shows html

主に参考したサイト：
Beautiful Soup 4によるスクレイピングの基礎
https://atmarkit.itmedia.co.jp/ait/articles/1910/18/news015.html

Chromeを表示させないで起動したい場合
https://prcmyself.com/selenium-scraping-with-browser-hidden



読み込んだHTMLからカード名を取得します。


Faced isssues:
P-> Imported card name twice which results that the deck contains more than 80 cards
C-> HTML mentions each card's name twice (probably to adapt multi-device interfaces)
S-> Creating For Loops that take out card name text from HTML but stop
when the number of imported cards reached the total number of cards in the deck

苦闘した点：
課題:カード名が２重で登場したため、取得したカード枚数が80枚以上となっていた。
原因:HTMLでカードが2度登場していた（おそらく複数デバイスに対応するためだろう）。
解決策:HTMLからカード名を取得し、デッキの枚数分読み込んだら読み込みを終了させるｆor文を作った。

In [154]:
deck_name = "DragonLink_PureVer1.0"
#how can i find text by multiple tag conditions??
# deck_name = soup.find('header', id= "broad_title").findChildren #デッキ名
# deck_name

num_title = soup.find_all('div', class_= "num_title") #メインデッキ合計
num_of_main = int(num_title[0].text)
num_of_extra = int(num_title[1].text)

print("メインデッキの枚数：{}".format(num_of_main))
print("エキストラデッキの枚数：{}".format(num_of_extra))


メインデッキの枚数：41
エキストラデッキの枚数：15


In [155]:
num_of_total = num_of_main + num_of_extra

card_list = soup.find_all('span', class_= "card_name") # デッキの全カード名をリストとして取得
cardList = [] # 配列を作成
num_list = soup.find_all('div', class_= "num") # カードの枚数を取得
numList = [] # 配列を作成

nm = 0
for n, m in zip(card_list,num_list):#spanタグのテキストデータを配列に格納
    sample_txt1 = n.text
    sample_txt2 = m.text.strip() #カードの枚数を取得する際にエラー。空白を消しておきます
    nm = nm + int(sample_txt2)
    if nm == num_of_total:
        break
    else:
        cardList.append(sample_txt1)
        numList.append(sample_txt2)

print("カードの種類:{}".format(cardList))
# print("カードの種類:{}".format(set(cardList))) #setは重複を削除できるが今回は使わないでよい
print(len(cardList))
print(numList)
print("カードの種類:{}".format(len(numList)))

カードの種類:{'ブレイクスルー・スキル', '混沌領域', '輝白竜 ワイバースター', '灰流うらら', 'ヴァレルロード・F・ドラゴン', '琰魔竜 レッド・デーモン・アビス', 'ツイン・トライアングル・ドラゴン', 'ヴァレット・リチャージャー', 'サンダー・ボルト', 'PSYフレーム・ドライバー', 'マグナヴァレット・ドラゴン', 'マスク・チェンジ・セカンド', 'クイック・リボルブ', '混沌魔龍 カオス・ルーラー', 'ヴァレルロード・S・ドラゴン', '暗黒竜 コラプサーペント', 'ヴァレット・シンクロン', 'ドラグニティナイト－ロムルス', '竜の渓谷', 'ノクトビジョン・ドラゴン', '王宮の勅命', '天球の聖刻印', 'アブソルーター・ドラゴン', 'トロイメア・ユニコーン', '煉獄龍 オーガ・ドラグーン', '墓穴の指名者', 'M・HERO ダーク・ロウ', '原始生命態ニビル', 'PSYフレームギア・γ', 'No.38 希望魁竜タイタニック・ギャラクシー', '星遺物の守護竜', '輝光竜セイファート', 'リボルブート・セクター', 'ハーピィの羽根帚', 'ヴァレルソード・ドラゴン', 'メタルヴァレット・ドラゴン', '龍の鏡', 'ストライカー・ドラゴン', '混源龍レヴィオニア', 'ヴァレット・トレーサー', '増殖するG'}
41
['1', '2', '3', '1', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '3', '1', '3', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
カードの種類:41


Now let Chrome to find each card information through web scraping!
This function automatically search card name in Yu-gi-oh wiki and determine if its card types (Monster, Magic, etc...)
Those values will be imported into dataframe.

クロームにカードの情報を探してもらいましょう！
下記の関数は、遊戯王Wikiでカード名を検索しカードの種類を取得するプログラムを書いています。取得した情報はデータフレームに落とします。

Faced isssues:
P-> Faced error not finding cards name individual page after key word was submited in the seach box
C-> Unknown 
S-> Skipped card that shows error and continue scraping from next card, yet leaves notes that which card has shown error

苦闘した点：
課題:検索後、指定したカードの固有ページリンクが見つからなかった。
原因:不明。
解決策:リンクが見つからないカードはスキップし、次のカードへ進む。ただ、見つからなかったカードの印は残しておく。

In [156]:
card_type = ["融合モンスター","シンクロモンスター","エクシーズモンスター","リンクモンスター","モンスター","魔法","罠"]
# Creating an empty Dataframe with column names only
df = pd.DataFrame(columns=['Card_name', 'Card_type', 'n_of_card'])
error_card = []

def card_register(index_number,keyword, n_of_card):
    print(keyword+"の情報を確認中。")
    # chromedriverの設定
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options) #hide Chrome
    # driver = webdriver.Chrome() #show Chrome
    driver.get('https://yugioh-wiki.net/')
    # time.sleep(1)  # Let the user actually see something!
    search_box = driver.find_element_by_name('word')
    search_box.send_keys(keyword)
    search_box.submit()
    # time.sleep(1)  # Let the user actually see something!
    try:
        page_link = driver.find_element_by_link_text('《'+keyword+'》')
    except NoSuchElementException:
        print(keyword+"はリンクが見つかりませんでした。\n ------") #to pick card that is not found
        error_card.append(keyword)
        df.loc[index_number]=[keyword, "unknown", n_of_card]
    else:
        url = page_link.get_attribute("href")
        # time.sleep(1)  # Let the user actually see something!
        res = requests.get(url)
        if res.status_code != requests.codes.ok:
            print('Error')
            exit(1)
        soup = bs4(res.text)
        tag = soup.find('span', class_= "tag").text
        tag_split_list = tag.split()
        for tag in tag_split_list:
            if tag in card_type:
                print("カードの種類:"+tag+"\n ------")
                cardType = tag
        df.loc[index_number]=[keyword, cardType, n_of_card]
        driver.quit()

In [157]:
nm = 0
for card_name, n in zip(cardList,numList):
    nm += 1
    print(nm)
    card_register(nm,card_name,n)

1
輝光竜セイファートの情報を確認中。
カードの種類:モンスター
 ------
2
アブソルーター・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
3
ヴァレット・トレーサーの情報を確認中。
カードの種類:モンスター
 ------
4
ヴァレット・シンクロンの情報を確認中。
カードの種類:モンスター
 ------
5
ヴァレット・リチャージャーの情報を確認中。
カードの種類:モンスター
 ------
6
灰流うららの情報を確認中。
カードの種類:モンスター
 ------
7
原始生命態ニビルの情報を確認中。
カードの種類:モンスター
 ------
8
混源龍レヴィオニアの情報を確認中。
カードの種類:モンスター
 ------
9
暗黒竜 コラプサーペントの情報を確認中。
カードの種類:モンスター
 ------
10
輝白竜 ワイバースターの情報を確認中。
カードの種類:モンスター
 ------
11
マグナヴァレット・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
12
メタルヴァレット・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
13
増殖するGの情報を確認中。
増殖するGはリンクが見つかりませんでした。
 ------
14
ノクトビジョン・ドラゴンの情報を確認中。
カードの種類:モンスター
 ------
15
PSYフレームギア・γの情報を確認中。
PSYフレームギア・γはリンクが見つかりませんでした。
 ------
16
PSYフレーム・ドライバーの情報を確認中。
PSYフレーム・ドライバーはリンクが見つかりませんでした。
 ------
17
リボルブート・セクターの情報を確認中。
カードの種類:魔法
 ------
18
クイック・リボルブの情報を確認中。
カードの種類:魔法
 ------
19
竜の渓谷の情報を確認中。
カードの種類:魔法
 ------
20
混沌領域の情報を確認中。
カードの種類:魔法
 ------
21
サンダー・ボルトの情報を確認中。
カードの種類:魔法
 ------
22
ハーピィの羽根帚の情報を確認中。
カードの種類:魔法
 ------
23
龍の鏡の情報を確認中。
カードの種類:魔法
 ------
24
星遺物の守護竜の情報を確

In [158]:
display(df)
print("エラーが出たカードです。")
print(error_card)

,Card_name,Card_type,Card_number
1,輝光竜セイファート,モンスター,1
2,アブソルーター・ドラゴン,モンスター,2
3,ヴァレット・トレーサー,モンスター,3
4,ヴァレット・シンクロン,モンスター,1
5,ヴァレット・リチャージャー,モンスター,1
6,灰流うらら,モンスター,2
7,原始生命態ニビル,モンスター,1
8,混源龍レヴィオニア,モンスター,1
9,暗黒竜 コラプサーペント,モンスター,1
10,輝白竜 ワイバースター,モンスター,2


エラーが出たカードです。
['増殖するG', 'PSYフレームギア・γ', 'PSYフレーム・ドライバー', 'M・HERO ダーク・ロウ', 'ヴァレルロード・S・ドラゴン', 'ヴァレルロード・F・ドラゴン', 'No.38 希望魁竜タイタニック・ギャラクシー', '琰魔竜 レッド・デーモン・アビス']


Divide card list into main deck and extra deck based on card types.

メインデッキと、EXデッキで表をわけたいのでdfをCard_typeに沿って分割します。

In [171]:
main_df = df[df['Card_type'].isin(["モンスター","魔法","罠"])]
extra_df = df[df['Card_type'].isin(["融合モンスター","シンクロモンスター","エクシーズモンスター","リンクモンスター"])]
print("Main Deck:")
display(main_df)
print("Extra Deck")
display(extra_df)

,Card_name,Card_type,Card_number
1,輝光竜セイファート,モンスター,1
2,アブソルーター・ドラゴン,モンスター,2
3,ヴァレット・トレーサー,モンスター,3
4,ヴァレット・シンクロン,モンスター,1
5,ヴァレット・リチャージャー,モンスター,1
6,灰流うらら,モンスター,2
7,原始生命態ニビル,モンスター,1
8,混源龍レヴィオニア,モンスター,1
9,暗黒竜 コラプサーペント,モンスター,1
10,輝白竜 ワイバースター,モンスター,2


,Card_name,Card_type,Card_number
30,ストライカー・ドラゴン,リンクモンスター,2
31,ヴァレルソード・ドラゴン,リンクモンスター,1
34,煉獄龍 オーガ・ドラグーン,シンクロモンスター,1
35,ツイン・トライアングル・ドラゴン,リンクモンスター,1
37,天球の聖刻印,リンクモンスター,1
38,ドラグニティナイト－ロムルス,リンクモンスター,1


Export two dataframes with csv!

最後にCSVで出力しましょう！

In [172]:
main_df.to_csv('{}_main.csv'.format(deck_name))
extra_df.to_csv('{}_extra.csv'.format(deck_name))